# Setup

In [2]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os
from pathlib import Path

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# To get smooth animations
import matplotlib.animation as animation
mpl.rc('animation', html='jshtml')

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "chapter_17"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
MODELS_PATH = os.path.join(PROJECT_ROOT_DIR, "models", CHAPTER_ID)
LOGS_PATH = os.path.join(PROJECT_ROOT_DIR, "logs", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)
os.makedirs(MODELS_PATH, exist_ok=True)
os.makedirs(LOGS_PATH, exist_ok=True)

In [3]:
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# 19.1 TensorFlowモデルのサーブ

### 19.1.1 TF Servingの使い方

In [6]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.mnist.load_data()
X_train_full = X_train_full[..., np.newaxis].astype(np.float32) / 255.
X_test = X_test[..., np.newaxis].astype(np.float32) / 255.
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_new = X_test[:3]

In [4]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28, 1]),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-2),
              metrics=["accuracy"])
model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
   1/1719 [..............................] - ETA: 3:54 - loss: 2.4127 - accuracy: 0.0312

2022-05-14 16:05:40.643813: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1719/1719 [==============================] - 1s 495us/step - loss: 0.7012 - accuracy: 0.8241 - val_loss: 0.3715 - val_accuracy: 0.9024
Epoch 2/10
1719/1719 [==============================] - 1s 443us/step - loss: 0.3536 - accuracy: 0.9020 - val_loss: 0.2990 - val_accuracy: 0.9144
Epoch 3/10
1719/1719 [==============================] - 1s 436us/step - loss: 0.3036 - accuracy: 0.9145 - val_loss: 0.2651 - val_accuracy: 0.9272
Epoch 4/10
1719/1719 [==============================] - 1s 438us/step - loss: 0.2736 - accuracy: 0.9231 - val_loss: 0.2436 - val_accuracy: 0.9334
Epoch 5/10
1719/1719 [==============================] - 1s 433us/step - loss: 0.2509 - accuracy: 0.9296 - val_loss: 0.2257 - val_accuracy: 0.9364
Epoch 6/10
1719/1719 [==============================] - 1s 438us/step - loss: 0.2322 - accuracy: 0.9350 - val_loss: 0.2121 - val_accuracy: 0.9396
Epoch 7/10
1719/1719 [==============================] - 1s 437us/step - loss: 0.2161 - accuracy: 0.9401 - val_loss: 0.1970 - val_accura

In [5]:
np.round(model.predict(X_new), 2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.99, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.97, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]],
      dtype=float32)

In [6]:
model_version = "0001"
model_name = "my_mnist_model"
model_path = os.path.join(model_name, model_version)
model_path

'my_mnist_model/0001'

In [7]:
!rm -rf {model_name}

In [8]:
tf.saved_model.save(model, model_path)

INFO:tensorflow:Assets written to: my_mnist_model/0001/assets


2022-05-14 16:07:20.239533: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


- ディレクトリ構造

In [9]:
for root, dirs, files in os.walk(model_name):
    indent = '    ' * root.count(os.sep)
    print('{}{}/'.format(indent, os.path.basename(root)))
    for filename in files:
        print('{}{}'.format(indent + '    ', filename))

my_mnist_model/
    0001/
        saved_model.pb
        variables/
            variables.data-00000-of-00001
            variables.index
        assets/


In [10]:
!saved_model_cli show --dir {model_path}

The given SavedModel contains the following tag-sets:
'serve'


In [11]:
!saved_model_cli show --dir {model_path} --tag_set serve

The given SavedModel MetaGraphDef contains SignatureDefs with the following keys:
SignatureDef key: "__saved_model_init_op"
SignatureDef key: "serving_default"


In [12]:
!saved_model_cli show --dir {model_path} --tag_set serve \
                      --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['flatten_input'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 28, 28, 1)
      name: serving_default_flatten_input:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['dense_1'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 10)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


In [13]:
!saved_model_cli show --dir {model_path} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['flatten_input'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 28, 28, 1)
        name: serving_default_flatten_input:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_1'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 10)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict

Defined Functions:
  Function Name: '__call__'
    Option #1
      Callable with:
        Argument #1
          inputs

In [14]:
np.save("my_mnist_tests.npy", X_new)

In [15]:
input_name = model.input_names[0]
input_name

'flatten_input'

- `saved_model_cli`を使って予測をしてみる

In [16]:
!saved_model_cli run --dir {model_path} --tag_set serve \
                     --signature_def serving_default    \
                     --inputs {input_name}=my_mnist_tests.npy

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from my_mnist_model/0001/variables/variables
2022-05-14 16:13:23.981552: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
Result for output key dense_1:
[[1.14598195e-04 1.51669241e-07 9.84688057e-04 2.77769845e-03
  3.77231186e-06 7.65922669e-05 3.92659985e-08 9.95557249e-01
  5.34842984e-05 4.31768305e-04]
 [8.21129070e-04 3.57630452e-05 9.88181412e-01 7.10086711e-03
  1.30191054e-07 2.35270432e-04 2.58035376e-03 9.77156245e-10
  1.04506174e-03 8.85531719e-08]
 [4.44632788e-05 9.70311999e-01 9.04908963e-03 2.26034364e-03
  4.85803670e-04 2.87339464e-03 2.27270857e-03 8.36258940e-03
  4.04201448e-03 2.97750492e-04]]


- 上記結果を見やすくすると以下（`np.round(model.predict(X_new), 2)`と一致するのが確認できる）

In [17]:
np.round([[1.1347984e-04, 1.5187356e-07, 9.7032893e-04, 2.7640699e-03, 3.7826971e-06,
           7.6876910e-05, 3.9140293e-08, 9.9559116e-01, 5.3502394e-05, 4.2665208e-04],
          [8.2443521e-04, 3.5493889e-05, 9.8826385e-01, 7.0466995e-03, 1.2957400e-07,
           2.3389691e-04, 2.5639210e-03, 9.5886099e-10, 1.0314899e-03, 8.7952529e-08],
          [4.4693781e-05, 9.7028232e-01, 9.0526715e-03, 2.2641101e-03, 4.8766597e-04,
           2.8800720e-03, 2.2714981e-03, 8.3753867e-03, 4.0439744e-03, 2.9759688e-04]], 2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.99, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.97, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]])

- Dockerを使う場合は以下を実行

```
docker pull tensorflow/serving

export ML_PATH=$HOME/ml # or wherever this project is
docker run -it --rm -p 8500:8500 -p 8501:8501 \
   -v "$ML_PATH/my_mnist_model:/models/my_mnist_model" \
   -e MODEL_NAME=my_mnist_model \
   tensorflow/serving
```

- `tensorflow_model_server`がインストールされている場合

In [19]:
# os.environ["MODEL_DIR"] = os.path.split(os.path.abspath(model_path))[0]

In [20]:
# %%bash --bg
# nohup tensorflow_model_server \
#      --rest_api_port=8501 \
#      --model_name=my_mnist_model \
#      --model_base_path="${MODEL_DIR}" >server.log 2>&1

In [22]:
# !tail server.log

In [23]:
import json

input_data_json = json.dumps({
    "signature_name": "serving_default",
    "instances": X_new.tolist(),
})

In [24]:
repr(input_data_json)[:1500] + "..."

'\'{"signature_name": "serving_default", "instances": [[[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]], [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]], [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]], [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]], [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0

- REST API

In [30]:
# import requests

# SERVER_URL = 'http://localhost:8501/v1/models/my_mnist_model:predict'
# response = requests.post(SERVER_URL, data=input_data_json)
# response.raise_for_status() # raise an exception in case of error
# response = response.json()

In [ ]:
# y_proba = np.array(response["predictions"])
# y_proba.round(2)

- gRPC API

In [31]:
# from tensorflow_serving.apis.predict_pb2 import PredictRequest

# request = PredictRequest()
# request.model_spec.name = model_name
# request.model_spec.signature_name = "serving_default"
# input_name = model.input_names[0]
# request.inputs[input_name].CopyFrom(tf.make_tensor_proto(X_new))

In [32]:
# import grpc
# from tensorflow_serving.apis import prediction_service_pb2_grpc

# channel = grpc.insecure_channel('localhost:8500')
# predict_service = prediction_service_pb2_grpc.PredictionServiceStub(channel)
# response = predict_service.Predict(request, timeout=10.0)

In [33]:
# response

In [34]:
# output_name = model.output_names[0]
# outputs_proto = response.outputs[output_name]
# y_proba = tf.make_ndarray(outputs_proto)
# y_proba.round(2)

- 新しいバージョンのモデルのデプロイ

In [35]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28, 1]),
    keras.layers.Dense(50, activation="relu"),
    keras.layers.Dense(50, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-2),
              metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 1s 445us/step - loss: 0.7039 - accuracy: 0.8056 - val_loss: 0.3418 - val_accuracy: 0.9042
Epoch 2/10
1719/1719 [==============================] - 1s 397us/step - loss: 0.3204 - accuracy: 0.9082 - val_loss: 0.2674 - val_accuracy: 0.9242
Epoch 3/10
1719/1719 [==============================] - 1s 399us/step - loss: 0.2650 - accuracy: 0.9235 - val_loss: 0.2227 - val_accuracy: 0.9366
Epoch 4/10
1719/1719 [==============================] - 1s 461us/step - loss: 0.2318 - accuracy: 0.9330 - val_loss: 0.2032 - val_accuracy: 0.9432
Epoch 5/10
1719/1719 [==============================] - 1s 425us/step - loss: 0.2088 - accuracy: 0.9398 - val_loss: 0.1831 - val_accuracy: 0.9478
Epoch 6/10
1719/1719 [==============================] - 1s 401us/step - loss: 0.1907 - accuracy: 0.9446 - val_loss: 0.1740 - val_accuracy: 0.9498
Epoch 7/10
1719/1719 [==============================] - 1s 401us/step - loss: 0.1755 - accuracy: 0.9491 - val_loss: 0.1604 -

In [36]:
model_version = "0002"
model_name = "my_mnist_model"
model_path = os.path.join(model_name, model_version)
model_path

'my_mnist_model/0002'

In [37]:
tf.saved_model.save(model, model_path)

INFO:tensorflow:Assets written to: my_mnist_model/0002/assets


In [38]:
for root, dirs, files in os.walk(model_name):
    indent = '    ' * root.count(os.sep)
    print('{}{}/'.format(indent, os.path.basename(root)))
    for filename in files:
        print('{}{}'.format(indent + '    ', filename))

my_mnist_model/
    0002/
        saved_model.pb
        variables/
            variables.data-00000-of-00001
            variables.index
        assets/
    0001/
        saved_model.pb
        variables/
            variables.data-00000-of-00001
            variables.index
        assets/


In [39]:
# import requests

# SERVER_URL = 'http://localhost:8501/v1/models/my_mnist_model:predict'
            
# response = requests.post(SERVER_URL, data=input_data_json)
# response.raise_for_status()
# response = response.json()

In [40]:
# y_proba = np.array(response["predictions"])
# y_proba.round(2)

### 19.1.2 GCP AI Platform上での予測サービスの作成

### 19.1.3 予測サービスの使い方

In [41]:
project_id = ""

In [42]:
# import googleapiclient.discovery

# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "my_service_account_private_key.json"
# model_id = "my_mnist_model"
# model_path = "projects/{}/models/{}".format(project_id, model_id)
# model_path += "/versions/v0001/" # if you want to run a specific version
# ml_resource = googleapiclient.discovery.build("ml", "v1").projects()

In [43]:
# def predict(X):
#     input_data_json = {"signature_name": "serving_default",
#                        "instances": X.tolist()}
#     request = ml_resource.predict(name=model_path, body=input_data_json)
#     response = request.execute()
#     if "error" in response:
#         raise RuntimeError(response["error"])
#     return np.array([pred[output_name] for pred in response["predictions"]])

In [44]:
# Y_probas = predict(X_new)
# np.round(Y_probas, 2)

# 19.2 モバイル/組み込みデバイスへのモデルのデプロイ

# 19.3 GPUを使った計算のスピードアップ

### 19.3.1 自前のGPU

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
tf.test.gpu_device_name()

In [ ]:
tf.test.is_built_with_cuda()

In [ ]:
from tensorflow.python.client.device_lib import list_local_devices

devices = list_local_devices()
devices

### 19.3.2 GPU装着仮想マシン

### 19.3.3 Colaboratory

### 19.3.4 GPU RAMの管理

### 19.3.5 オペレーション、変数のデバイスへの配置

### 19.3.6 複数のデバイスにまたがる並列実行

# 19.4 複数のデバイスによるモデルの訓練

### 19.4.1 モデル並列

### 19.4.2 データ並列

### 19.4.3 Distribution Strategy APIを使った大規模な訓練

In [4]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

In [5]:
def create_model():
    return keras.models.Sequential([
        keras.layers.Conv2D(filters=64, kernel_size=7, activation="relu",
                            padding="same", input_shape=[28, 28, 1]),
        keras.layers.MaxPooling2D(pool_size=2),
        keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu",
                            padding="same"), 
        keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu",
                            padding="same"),
        keras.layers.MaxPooling2D(pool_size=2),
        keras.layers.Flatten(),
        keras.layers.Dense(units=64, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(units=10, activation='softmax'),
    ])

In [7]:
batch_size = 100
model = create_model()
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-2),
              metrics=["accuracy"])
model.fit(X_train, y_train, epochs=10,
          validation_data=(X_valid, y_valid), batch_size=batch_size)

2022-05-21 13:54:05.711744: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/10
550/550 [==============================] - 64s 116ms/step - loss: 1.2576 - accuracy: 0.5984 - val_loss: 0.3428 - val_accuracy: 0.9006
Epoch 2/10
550/550 [==============================] - 70s 127ms/step - loss: 0.4479 - accuracy: 0.8641 - val_loss: 0.1954 - val_accuracy: 0.9452
Epoch 3/10
550/550 [==============================] - 73s 133ms/step - loss: 0.3084 - accuracy: 0.9090 - val_loss: 0.1348 - val_accuracy: 0.9616
Epoch 4/10
550/550 [==============================] - 79s 143ms/step - loss: 0.2415 - accuracy: 0.9290 - val_loss: 0.1055 - val_accuracy: 0.9708
Epoch 5/10
550/550 [==============================] - 78s 142ms/step - loss: 0.2033 - accuracy: 0.9421 - val_loss: 0.0869 - val_accuracy: 0.9752
Epoch 6/10
550/550 [==============================] - 78s 142ms/step - loss: 0.1814 - accuracy: 0.9467 - val_loss: 0.0796 - val_accuracy: 0.9780
Epoch 7/10
550/550 [==============================] - 79s 143ms/step - loss: 0.1606 - accuracy: 0.9522 - val_loss: 0.0745 - val_ac

In [9]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

distribution = tf.distribute.MirroredStrategy()

with distribution.scope():
    model = create_model()
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer=keras.optimizers.SGD(learning_rate=1e-2),
                  metrics=["accuracy"])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [10]:
batch_size = 100 # must be divisible by the number of workers
model.fit(X_train, y_train, epochs=10,
          validation_data=(X_valid, y_valid), batch_size=batch_size)

2022-05-21 14:06:54.061019: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:766] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: -2
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_25398"
    }
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\021FlatMapDataset:57"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 100
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this dataset. Y

Epoch 1/10
550/550 [==============================] - ETA: 0s - loss: 1.2576 - accuracy: 0.5984

2022-05-21 14:08:09.364100: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:766] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: -2
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_28726"
    }
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\021FlatMapDataset:79"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 100
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this dataset. Y

550/550 [==============================] - 78s 142ms/step - loss: 1.2576 - accuracy: 0.5984 - val_loss: 0.3428 - val_accuracy: 0.9006
Epoch 2/10
550/550 [==============================] - ETA: 0s - loss: 0.4479 - accuracy: 0.8641

2022-05-21 14:09:29.588886: W tensorflow/core/framework/dataset.cc:744] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


...

550/550 [==============================] - 80s 146ms/step - loss: 0.1497 - accuracy: 0.9560 - val_loss: 0.0691 - val_accuracy: 0.9808
Epoch 9/10
550/550 [==============================] - ETA: 0s - loss: 0.1358 - accuracy: 0.9602

2022-05-21 14:18:51.692777: W tensorflow/core/framework/dataset.cc:744] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


550/550 [==============================] - 81s 147ms/step - loss: 0.1358 - accuracy: 0.9602 - val_loss: 0.0637 - val_accuracy: 0.9812
Epoch 10/10
550/550 [==============================] - ETA: 0s - loss: 0.1320 - accuracy: 0.9622

2022-05-21 14:20:12.468526: W tensorflow/core/framework/dataset.cc:744] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


550/550 [==============================] - 81s 147ms/step - loss: 0.1320 - accuracy: 0.9622 - val_loss: 0.0604 - val_accuracy: 0.9820


In [11]:
model.predict(X_new)

2022-05-21 14:20:19.558591: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:766] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: -2
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_57187"
    }
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\022FlatMapDataset:153"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this dataset. Y

array([[1.19315091e-09, 1.01480857e-09, 2.33716287e-06, 6.68458284e-08,
        8.75595429e-11, 1.31257574e-10, 1.11427062e-13, 9.99995112e-01,
        3.69021658e-09, 2.42146439e-06],
       [1.03919277e-07, 8.04047959e-05, 9.99919415e-01, 1.26099415e-08,
        8.55898685e-11, 1.27486532e-09, 3.81773049e-08, 3.24152261e-09,
        1.15491474e-07, 7.33474392e-10],
       [2.96316642e-07, 9.99883890e-01, 3.31762712e-05, 1.69169846e-06,
        4.47833809e-05, 6.61029048e-07, 1.47179935e-05, 1.04452129e-05,
        5.46047386e-06, 4.75413754e-06]], dtype=float32)

- カスタムトレーニング

In [12]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

K = keras.backend

distribution = tf.distribute.MirroredStrategy()

with distribution.scope():
    model = create_model()
    optimizer = keras.optimizers.SGD()

with distribution.scope():
    dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).repeat().batch(batch_size)
    input_iterator = distribution.make_dataset_iterator(dataset)
    
@tf.function
def train_step():
    def step_fn(inputs):
        X, y = inputs
        with tf.GradientTape() as tape:
            Y_proba = model(X)
            loss = K.sum(keras.losses.sparse_categorical_crossentropy(y, Y_proba)) / batch_size

        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        return loss

    per_replica_losses = distribution.experimental_run(step_fn, input_iterator)
    mean_loss = distribution.reduce(tf.distribute.ReduceOp.SUM,
                                    per_replica_losses, axis=None)
    return mean_loss

n_epochs = 10
with distribution.scope():
    input_iterator.initialize()
    for epoch in range(n_epochs):
        print("Epoch {}/{}".format(epoch + 1, n_epochs))
        for iteration in range(len(X_train) // batch_size):
            print("\rLoss: {:.3f}".format(train_step().numpy()), end="")
        print()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Instructions for updating:
Use the iterator's `initializer` property instead.


2022-05-21 15:07:56.703434: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:766] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_FLOAT
      type: DT_UINT8
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 55000
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\026TensorSliceDataset:172"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 28
        }
        dim {
          size: 28
        }
        dim {
          size: 1
        }
      }
      shape {
      }
    }
  }
}

2022-05-21 15:07:56.884981: W tensorflow/core/framework/dataset.cc:744] Input of GeneratorDatasetOp::Dataset 

Epoch 1/10
Instructions for updating:
use run() instead
Loss: 0.379
Epoch 2/10
Loss: 0.301
Epoch 3/10
Loss: 0.286
Epoch 4/10
Loss: 0.294
Epoch 5/10
Loss: 0.300
Epoch 6/10
Loss: 0.306
Epoch 7/10
Loss: 0.308
Epoch 8/10
Loss: 0.305
Epoch 9/10
Loss: 0.301
Epoch 10/10
Loss: 0.296


### 19.4.4 TenssorFlowクラスタによるモデルの訓練

In [13]:
cluster_spec = {
    "worker": [
        "machine-a.example.com:2222",  # /job:worker/task:0
        "machine-b.example.com:2222"   # /job:worker/task:1
    ],
    "ps": ["machine-c.example.com:2222"] # /job:ps/task:0
}

In [14]:
import os
import json

os.environ["TF_CONFIG"] = json.dumps({
    "cluster": cluster_spec,
    "task": {"type": "worker", "index": 1}
})
os.environ["TF_CONFIG"]

'{"cluster": {"worker": ["machine-a.example.com:2222", "machine-b.example.com:2222"], "ps": ["machine-c.example.com:2222"]}, "task": {"type": "worker", "index": 1}}'

In [15]:
import tensorflow as tf

resolver = tf.distribute.cluster_resolver.TFConfigClusterResolver()
resolver.cluster_spec()

ClusterSpec({'ps': ['machine-c.example.com:2222'], 'worker': ['machine-a.example.com:2222', 'machine-b.example.com:2222']})

In [16]:
resolver.task_type

'worker'

In [17]:
resolver.task_id

1

- ローカルマシン上で動作する2つのワーカータスクを持つシンプルなクラスタを実行

In [21]:
# 学習コード準備

In [18]:
%%writefile my_mnist_multiworker_task.py

import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
import time

# At the beginning of the program
distribution = tf.distribute.MultiWorkerMirroredStrategy()

resolver = tf.distribute.cluster_resolver.TFConfigClusterResolver()
print("Starting task {}{}".format(resolver.task_type, resolver.task_id))

# Only worker #0 will write checkpoints and log to TensorBoard
if resolver.task_id == 0:
    root_logdir = os.path.join(os.curdir, "my_mnist_multiworker_logs")
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    run_dir = os.path.join(root_logdir, run_id)
    callbacks = [
        keras.callbacks.TensorBoard(run_dir),
        keras.callbacks.ModelCheckpoint("my_mnist_multiworker_model.h5",
                                        save_best_only=True),
    ]
else:
    callbacks = []

# Load and prepare the MNIST dataset
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.mnist.load_data()
X_train_full = X_train_full[..., np.newaxis] / 255.
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

with distribution.scope():
    model = keras.models.Sequential([
        keras.layers.Conv2D(filters=64, kernel_size=7, activation="relu",
                            padding="same", input_shape=[28, 28, 1]),
        keras.layers.MaxPooling2D(pool_size=2),
        keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu",
                            padding="same"), 
        keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu",
                            padding="same"),
        keras.layers.MaxPooling2D(pool_size=2),
        keras.layers.Flatten(),
        keras.layers.Dense(units=64, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(units=10, activation='softmax'),
    ])
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer=keras.optimizers.SGD(learning_rate=1e-2),
                  metrics=["accuracy"])

model.fit(X_train, y_train, validation_data=(X_valid, y_valid),
          epochs=10, callbacks=callbacks)

Writing my_mnist_multiworker_task.py


- メモリ不足回避のため、各ワーカーに異なるGPUを割り当てる

In [22]:
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [23]:
import subprocess

cluster_spec = {"worker": ["127.0.0.1:9901", "127.0.0.1:9902"]}

for index, worker_address in enumerate(cluster_spec["worker"]):
    os.environ["TF_CONFIG"] = json.dumps({
        "cluster": cluster_spec,
        "task": {"type": "worker", "index": index}
    })
    subprocess.Popen("python my_mnist_multiworker_task.py", shell=True)

2022-05-22 21:17:46.232529: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:272] Initialize GrpcChannelCache for job worker -> {0 -> 127.0.0.1:9901, 1 -> 127.0.0.1:9902}
2022-05-22 21:17:46.232529: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:272] Initialize GrpcChannelCache for job worker -> {0 -> 127.0.0.1:9901, 1 -> 127.0.0.1:9902}
2022-05-22 21:17:46.235969: I tensorflow/core/distributed_runtime/rpc/grpc_server_lib.cc:427] Started server with target: grpc://127.0.0.1:9901
2022-05-22 21:17:46.235971: I tensorflow/core/distributed_runtime/rpc/grpc_server_lib.cc:427] Started server with target: grpc://127.0.0.1:9902
2022-05-22 21:17:47.031935: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:766] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "Pref

Starting task worker1
Epoch 1/10
   1/1719 [..............................] - ETA: 1:09:20 - loss: 2.3033 - accuracy: 0.0938Starting task worker0
Epoch 1/10
1719/1719 [==============================] - ETA: 0s - loss: 0.7847 - accuracy: 0.7497

2022-05-22 21:19:05.599614: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:766] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: -2
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_20279"
    }
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\021FlatMapDataset:30"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this dataset. Yo

1719/1719 [==============================] - 81s 46ms/step - loss: 0.7847 - accuracy: 0.7497 - val_loss: 0.1498 - val_accuracy: 0.9582
Epoch 2/10
1650/1719 [===========================>..] - ETA: 3s - loss: 0.2321 - accuracy: 0.9326

In [24]:
%load_ext tensorboard
%tensorboard --logdir=./my_mnist_multiworker_logs --port=6006

1654/1719 [===========================>..] - ETA: 3s - loss: 0.2321 - accuracy: 0.9326

1718/1719 [============================>.] - ETA: 0s - loss: 0.2302 - accuracy: 0.9333

2022-05-22 21:20:33.354467: W tensorflow/core/framework/dataset.cc:744] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.
2022-05-22 21:20:33.440495: W tensorflow/core/framework/dataset.cc:744] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


1719/1719 [==============================] - 88s 51ms/step - loss: 0.2301 - accuracy: 0.9333 - val_loss: 0.0826 - val_accuracy: 0.9766
Epoch 3/10
1718/1719 [============================>.] - ETA: 0s - loss: 0.1687 - accuracy: 0.9503

2022-05-22 21:22:09.463836: W tensorflow/core/framework/dataset.cc:744] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.
2022-05-22 21:22:09.465600: W tensorflow/core/framework/dataset.cc:744] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


1719/1719 [==============================] - 96s 56ms/step - loss: 0.1686 - accuracy: 0.9504 - val_loss: 0.0661 - val_accuracy: 0.9804
Epoch 4/10
1718/1719 [============================>.] - ETA: 0s - loss: 0.1374 - accuracy: 0.9599

2022-05-22 21:23:52.833384: W tensorflow/core/framework/dataset.cc:744] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.
2022-05-22 21:23:52.920145: W tensorflow/core/framework/dataset.cc:744] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


1719/1719 [==============================] - 103s 60ms/step - loss: 0.1374 - accuracy: 0.9599 - val_loss: 0.0571 - val_accuracy: 0.9838
Epoch 5/10
 474/1719 [=======>......................] - ETA: 1:12 - loss: 0.1233 - accuracy: 0.9635

In [29]:
from tensorflow import keras

model = keras.models.load_model("my_mnist_multiworker_model.h5")
Y_pred = model.predict(X_new)
np.argmax(Y_pred, axis=-1)

array([7, 2, 1])

### 19.4.5 Google Cloud AI Platformでの大規模な訓練ジョブの実行

### 19.4.6 AI Platform上でのブラックボックス型のハイパーパラメータ調整サービス

# 19. 演習問題

### 19.5.9 

In [30]:
# 省略

### 19.5.10

In [31]:
# 省略

### 19.5.11

In [32]:
# 省略